In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-07 11:34:36--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-07 11:34:36 (43.4 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import minsearch

In [4]:
import json

In [4]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-06-07 11:36:38--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.008s  

2025-06-07 11:36:38 (79.9 MB/s) - ‘documents.json’ saved [658332/658332]



In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
index.fit(documents)

In [1]:
import openai
from openai import OpenAI

In [2]:
client = OpenAI()

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It depends on the specific course and institution's policies. Some courses allow late enrollment within a certain time frame, often with a penalty or additional requirements. It's best to contact the course administrator or check the institution's website for details on their enrollment policies and any deadlines."

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
    )

    return response.choices[0].message.content

In [36]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
print(answer)

Running Apache Kafka involves several steps, including setting up Zookeeper (since Kafka depends on it for its operations) and starting the Kafka server. Here's a step-by-step guide to get you started:

### Pre-requisites
- Ensure you have Java installed on your machine, preferably version 8 or later.
- Download Apache Kafka from the [official website](https://kafka.apache.org/downloads).

### Steps to Run Kafka

1. **Extract Kafka:**
   - Once downloaded, extract the Kafka binary files to a directory on your system.

2. **Start Zookeeper:**
   - Kafka requires Zookeeper to manage distributed brokers. Kafka binary includes a Zookeeper instance for you to use.
   - Open a terminal window, navigate to the Kafka directory, and run the following command:

     ```bash
     bin/zookeeper-server-start.sh config/zookeeper.properties
     ```

   - This starts a Zookeeper instance with the default configuration.

3. **Start Kafka Broker:**
   - Open another terminal window, navigate to the Kaf

In [25]:
from elasticsearch import Elasticsearch


In [26]:
es_client = Elasticsearch('http://localhost:9200')

In [27]:
es_client.info()

ObjectApiResponse({'name': '7157e1ea22b5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'IZEgnO0rQaKvDu0-lVt4lA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [29]:
index_settings  = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [31]:
from tqdm.auto import tqdm

In [34]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [38]:
query 

'how do I run kafka?'

In [41]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [42]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [44]:
print(answer)

Running Apache Kafka involves several steps, including setting up Zookeeper (since Kafka depends on it for its operations) and starting the Kafka server. Here's a step-by-step guide to get you started:

### Pre-requisites
- Ensure you have Java installed on your machine, preferably version 8 or later.
- Download Apache Kafka from the [official website](https://kafka.apache.org/downloads).

### Steps to Run Kafka

1. **Extract Kafka:**
   - Once downloaded, extract the Kafka binary files to a directory on your system.

2. **Start Zookeeper:**
   - Kafka requires Zookeeper to manage distributed brokers. Kafka binary includes a Zookeeper instance for you to use.
   - Open a terminal window, navigate to the Kafka directory, and run the following command:

     ```bash
     bin/zookeeper-server-start.sh config/zookeeper.properties
     ```

   - This starts a Zookeeper instance with the default configuration.

3. **Start Kafka Broker:**
   - Open another terminal window, navigate to the Kaf